<a href="https://colab.research.google.com/github/t3r4n/Bayesiana/blob/main/Bayesiana_modelo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import arviz as az
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import pymc as pm

In [ ]:
df = pd.read_excel('/content/BASE_BASE (1).xlsx')

#**FILTRADO DE VARIABLES DE VALOR**

In [ ]:
selected_columns = ['SEXO', 'EDAD', 'PAIS_RESIDENCIA', 'PROPOSITO',
       'IMPORTANCIA_OFERTA_ALOJATIVA', 'MEDIDA_LIMPIEZA',
       'MEDIDA_FACILIDAD_TP', 'NOCHES', 'PERSONAS_TOTAL', 'COSTE_ALOJ_EUROS',
       'HORAS_DIA', 'IMPORTANCIA_SEGURIDAD', 'IMPORTANCIA_PRECIO',
       'ANTELACION_VIAJE', 'CANAL_VISITAS_ANTERIORES',
       'CANAL_AMIGOS_FAMILIARES', 'CANAL_INTERNET_REDES_SOCIALES',
       'CANAL_TOUROPERADOR_AGENCIA_VIAJE', 'COMPANNIA_SOLO', 'ALOJ_CATEG',
       'INGRESOS', 'COMPRA_ALOJ_TTOO_AGENCIA_VIAJES']
filtered_df = df[selected_columns]
filtered_df

In [ ]:
filtered_df['PAIS_RESIDENCIA'] = filtered_df['PAIS_RESIDENCIA'].astype('category').cat.codes
filtered_df['ALOJ_CATEG'] = filtered_df['ALOJ_CATEG'].astype(int)
filtered_df['PAIS_RESIDENCIA']

#**TRATAMIENTO DE LOS DATOS**

In [ ]:
X = filtered_df[['NOCHES', 'COSTE_ALOJ_EUROS', 'MEDIDA_LIMPIEZA', 'PROPOSITO',
                 'INGRESOS', 'SEXO', 'EDAD', 'PERSONAS_TOTAL',
                 'CANAL_INTERNET_REDES_SOCIALES', 'CANAL_TOUROPERADOR_AGENCIA_VIAJE',
                 'CANAL_AMIGOS_FAMILIARES', 'CANAL_VISITAS_ANTERIORES',
                 'IMPORTANCIA_SEGURIDAD', 'IMPORTANCIA_PRECIO', 'IMPORTANCIA_OFERTA_ALOJATIVA']].values

y = filtered_df['ALOJ_CATEG'].values
pais_idx = filtered_df['PAIS_RESIDENCIA'].values

# Estandarizar las variables predictoras
X_scaled = (X - X.mean(axis=0)) / X.std(axis=0)


# Revisar dimensiones
print(f"Dimensiones de las variables predictoras (X_scaled): {X_scaled.shape}")
print(f"Dimensiones de la variable objetivo (y): {y.shape}")
print(f"Índices por país (pais_idx): {pais_idx.shape}")

In [ ]:
coords = {
    "PAIS": np.sort(filtered_df['PAIS_RESIDENCIA'].unique()),  # Países como coordenadas
    "Observaciones": np.arange(filtered_df.shape[0])  # Índices para observaciones
}

In [ ]:
# Verificar los datos
# 1. Confirmar que la variable objetivo (y) es binaria
assert set(y) <= {0, 1}, "La variable objetivo (ALOJ_CATEG) debe ser binaria (0 o 1)."

# 2. Verificar los índices por país
assert pais_idx.min() >= 0 and pais_idx.max() < len(coords["PAIS"]), "Los índices de países (pais_idx) están fuera de rango."

# 3. Estandarizar las variables predictoras
X_scaled = (X - X.mean(axis=0)) / X.std(axis=0)

In [ ]:
# Manejar valores extremos con log para COSTE_ALOJ_EUROS
X[:, 1] = np.log1p(X[:, 1])  # Aplicar transformación logarítmica

# Convertir X a DataFrame para manejar nulos
X_df = pd.DataFrame(X)

# Calcular la media de cada columna y rellenar los valores nulos
X_filled = X_df.fillna({col: X_df[col].mean() for col in X_df.columns}).values

# Escalar las variables predictoras
X_scaled = (X_filled - X_filled.mean(axis=0)) / X_filled.std(axis=0)

#**CREACIÓN DEL MODELO JERÁRQUICO**

In [ ]:
# Crear modelo jerárquico
with pm.Model(coords=coords) as aloj_model:

    # Priors globales
    intercept_global = pm.Normal("Intercept_Global", mu=0, sigma=10)  # Priors más amplios
    sigma_global = pm.HalfNormal("Sigma_Global", sigma=5)

    # Priors específicos para países
    intercept_pais = pm.Normal("Intercept_Pais", mu=intercept_global, sigma=sigma_global, dims="PAIS")

    # Coeficientes globales para las variables predictoras
    betas = pm.Normal("Betas", mu=0, sigma=10, shape=X_scaled.shape[1])

    # Construcción de la media
    mu = intercept_pais[pais_idx] + pm.math.dot(X_scaled, betas)

    # Likelihood (logit link)
    p = pm.math.sigmoid(mu)
    y_obs = pm.Bernoulli("ALOJ_CATEG", p=p, observed=y)

    # Muestreo MCMC con inicialización robusta
    trace = pm.sample(
        1000,
        return_inferencedata=True,
        cores=1,
        chains=2,
        target_accept=0.95,
        init="jitter+adapt_diag"
    )

##**priors**

In [ ]:
az.plot_posterior(trace, var_names=["Intercept_Global", "Sigma_Global", "Intercept_Pais"],
                  figsize=(22, 23))
plt.show()

##**Posterior**


In [ ]:
az.plot_trace(trace)
plt.tight_layout()
plt.show()

#**Organización para visaulizar betas**

In [ ]:
# Variables predictoras en orden
variables = ['NOCHES', 'COSTE_ALOJ_EUROS', 'MEDIDA_LIMPIEZA', 'PROPOSITO',
                 'INGRESOS', 'SEXO', 'EDAD', 'PERSONAS_TOTAL',
                 'CANAL_INTERNET_REDES_SOCIALES', 'CANAL_TOUROPERADOR_AGENCIA_VIAJE',
                 'CANAL_AMIGOS_FAMILIARES', 'CANAL_VISITAS_ANTERIORES',
                 'IMPORTANCIA_SEGURIDAD', 'IMPORTANCIA_PRECIO', 'IMPORTANCIA_OFERTA_ALOJATIVA']

# Extraer los valores posteriores de los betas
betas_trace = trace.posterior['Betas'].mean(dim=['chain', 'draw']).values

# Asignar cada beta a su variable
for var, beta in zip(variables, betas_trace):
    print(f"Beta para {var}: {beta}")

In [ ]:
# prompt: Quiero ver los paises en orden

# Obtener los códigos de país únicos y sus nombres correspondientes
pais_codes = filtered_df['PAIS_RESIDENCIA'].unique()
pais_names = df['PAIS_RESIDENCIA'].unique()

# Crear un diccionario para mapear los códigos de país a sus nombres
pais_mapping = {code: name for code, name in zip(pais_codes, pais_names)}

# Ordenar los países por su código (orden alfabético del nombre)
sorted_pais_codes = np.sort(pais_codes)

# Imprimir los países en orden
for code in sorted_pais_codes:
  print(f"Código de país: {code}, Nombre del país: {pais_mapping[code]}")

#**Interceptos por país**


In [ ]:
print(trace)
with aloj_model:
    posterior_predictive = pm.sample_posterior_predictive(trace, var_names=["ALOJ_CATEG"], random_seed=42)


In [ ]:
# Obtener los interceptos específicos por país
intercept_pais_summary = az.summary(trace, var_names=["Intercept_Pais"], hdi_prob=0.94)

# Crear un mapeo entre los códigos y los nombres de los países
unique_paises = filtered_df[['PAIS_RESIDENCIA']].drop_duplicates()
unique_paises['Codigo'] = unique_paises['PAIS_RESIDENCIA'].astype('category').cat.codes
pais_mapping = unique_paises.sort_values('Codigo')

# Asignar cada intercepto a su país correspondiente
intercept_pais_summary['Pais'] = pais_mapping['PAIS_RESIDENCIA'].values

# Mostrar el resumen de los interceptos por país
intercept_pais_summary